In [1]:
!git clone https://github.com/OSU-BMBL/SpaGFT

fatal: destination path 'SpaGFT' already exists and is not an empty directory.


In [2]:

!pip install -r SpaGFT/requirements.txt

In [3]:
import scanpy as sc

In [9]:
file_path_brain = '/kaggle/input/gi-input/Mouse_brain_cell_bin.h5ad'
file_path_embryo = '/kaggle/input/gi-input/E9.5_E1S1.MOSTA.h5ad'

In [10]:
anndata_brain=sc.read(file_path_brain)
anndata_embryo = sc.read(file_path_embryo)
anndata_embryo.var_names_make_unique()
# Add raw to anndata object
anndata_embryo.raw = anndata_embryo

Only considering the two last: ['.MOSTA', '.h5ad'].
Only considering the two last: ['.MOSTA', '.h5ad'].


/opt/conda/lib/python3.10/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


In [11]:
!python SpaGFT/setup.py install

/opt/conda/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/opt/conda/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://github.com/py

In [13]:
import SpaGFT as spg
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import os


sc.settings.verbosity = 1
sc.settings.set_figure_params(dpi=80, facecolor='white')

In [14]:
# QC
sc.pp.filter_genes(anndata_embryo, min_cells=10)
# Normalization
sc.pp.normalize_total(anndata_embryo, inplace=True)
sc.pp.log1p(anndata_embryo)

In [15]:
print(anndata_embryo.obsm['spatial'])

[[ 104. -147.]
 [ 105. -147.]
 [ 106. -147.]
 ...
 [ 112. -253.]
 [ 113. -253.]
 [ 114. -253.]]


In [35]:
# Import SpaGFT
import SpaGFT.gft as gft_module

# Access the determine_frequency_ratio function within the gft module
print(anndata_embryo.obsm)
(ratio_low, ratio_high) = gft_module.determine_frequency_ratio(anndata_embryo, ratio_neighbors=1,spatial_info='spatial')


AxisArrays with keys: spatial
Obatain the Laplacian matrix


In [36]:
gene_df = gft_module.detect_svg(anndata_embryo,
                         spatial_info='spatial',
                         ratio_low_freq=ratio_low,
                         ratio_high_freq=ratio_high,
                         ratio_neighbors=1,
                         filter_peaks=True,
                         S=6)
svg_list = gene_df[gene_df.cutoff_gft_score][gene_df.qvalue < 0.05].index.tolist()
print("The number of SVGs: ", len(svg_list))
print(svg_list[:20])

The precalculated low-frequency FMs are USED
The precalculated high-frequency FMs are USED
Graph Fourier Transform finished!
svg ranking could be found in adata.var['svg_rank']
The spatially variable genes judged by gft_score could be found 
          in adata.var['cutoff_gft_score']
Gene signals in frequency domain when detect svgs could be found
          in adata.varm['freq_domain_svg']
The number of SVGs:  2391
['Ckb', 'Hoxc5', 'Fabp7', 'Arg1', 'Myl7', 'En2', 'Rfx4', 'Otx2', 'Dlk1', 'Actc1', 'Myl4', 'Peg3', 'Msi1', 'Zic1', 'Hoxc8', 'Myl3', 'Hoxb3os', 'Meox1', 'Myh7', '2510009E07Rik']


In [38]:
anndata_brain.var_names_make_unique()
anndata_brain.raw = anndata_brain

In [39]:

sc.pp.filter_genes(anndata_brain, min_cells=10)

sc.pp.normalize_total(anndata_brain, inplace=True)
sc.pp.log1p(anndata_brain)

In [40]:
(ratio_low_brain, ratio_high_brain) = gft_module.determine_frequency_ratio(anndata_brain, ratio_neighbors=1,spatial_info='spatial')

Obatain the Laplacian matrix


In [43]:
gene_df_brain = gft_module.detect_svg(anndata_brain,
                         spatial_info='spatial',
                         ratio_low_freq=ratio_low_brain,
                         ratio_high_freq=ratio_high_brain,
                         ratio_neighbors=1,
                         filter_peaks=True,
                         S=6)
# S determines the  sensitivity of kneedle algorithm
# extract spaitally variable genes
svg_list_brain = gene_df_brain[gene_df_brain.cutoff_gft_score][gene_df_brain.qvalue < 0.05].index.tolist()
print("The number of SVGs: ", len(svg_list_brain))
# the top 20 SVGs
print(svg_list_brain[:20])

The precalculated low-frequency FMs are USED
The precalculated high-frequency FMs are USED
Graph Fourier Transform finished!
svg ranking could be found in adata.var['svg_rank']
The spatially variable genes judged by gft_score could be found 
          in adata.var['cutoff_gft_score']
Gene signals in frequency domain when detect svgs could be found
          in adata.varm['freq_domain_svg']
The number of SVGs:  2187
['Tcf7l2', 'Camk2n1', 'Nrgn', 'Ptms', 'Arpp19', 'Cabp1', 'Mef2c', 'Mbp', 'Atp1a1', 'Lamp5', 'Slc17a7', 'Satb2', 'Plp1', 'Tshz2', 'Phactr1', 'Egr1', 'Osbpl1a', 'Icam5', 'Gm28928', 'Sparc']


In [44]:
def save_genes_to_txt(genes, filename):
    with open(filename, 'w') as file:
        for gene in genes:
            file.write(gene + '\n')

embryo_filename = 'svg_genes_embryo_SpaGFT.txt'
brain_filename = 'svg_genes_brain_SpaGFT.txt'

save_genes_to_txt(svg_list, embryo_filename)

save_genes_to_txt(svg_list_brain, brain_filename)

print(f"Spatially variable genes for Mouse Embryo 9.5 saved to '{embryo_filename}'.")
print(f"Spatially variable genes for Mouse Brain saved to '{brain_filename}'.")

Spatially variable genes for Mouse Embryo 9.5 saved to 'svg_genes_embryo_SpaGFT.txt'.
Spatially variable genes for Mouse Brain saved to 'svg_genes_brain_SpaGFT.txt'.
